In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery
from matplotlib.ticker import PercentFormatter


from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by country
UK_spending_by_country = '''SELECT time_period_value, destination_country, spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg = 'All'
and destination_country != 'UNITED KINGDOM'
GROUP BY destination_country, time_period_value, spend
ORDER BY time_period_value, spend DESC'''


country_quarter_year_spend = bq.read_bq_table_sql(client, UK_spending_by_country)

country_quarter_year_spend['year'] = country_quarter_year_spend['time_period_value'].str[:4]

# Convert the 'year' column to numeric
country_quarter_year_spend['year'] = pd.to_numeric(country_quarter_year_spend['year'], errors='coerce')

country_quarter_year_spend

In [ ]:
completeness_matrix_country = (
country_quarter_year_spend.pivot_table(index='destination_country', columns='time_period_value', values='spend', aggfunc='size')
    .fillna(False)
    .astype(bool)
)

incomplete_rows_country = completeness_matrix_country[~completeness_matrix_country.all(axis=1)]
incomplete_rows_country

In [ ]:
# Filter data for 2019 and 2024
country_quarter_year_spend_2019_2024 = country_quarter_year_spend[country_quarter_year_spend['year'].isin([2019, 2024])]

# Group by year and category, summing the spend
country_year_spend_2019_2024 = country_quarter_year_spend_2019_2024.groupby(['year', 'destination_country'])['spend'].sum().reset_index()

# Pivot the data to have years as columns
pivot_country_year_spend_2019_2024 = country_year_spend_2019_2024.pivot(index='destination_country', columns='year', values='spend').fillna(0)

# Calculate percentage of total spend for each year
pivot_country_year_spend_2019_2024[2019] = pivot_country_year_spend_2019_2024[2019] / pivot_country_year_spend_2019_2024[2019].sum()
pivot_country_year_spend_2019_2024[2024] = pivot_country_year_spend_2019_2024[2024] / pivot_country_year_spend_2019_2024[2024].sum()

# Sort categories by 2024 spend
pivot_country_year_spend_2019_2024 = pivot_country_year_spend_2019_2024.sort_values(by=2024, ascending=False)

# Plotting
categories = pivot_country_year_spend_2019_2024.index.tolist()
spend_2024 = pivot_country_year_spend_2019_2024[2024].values
spend_2019 = pivot_country_year_spend_2019_2024[2019].values

x = np.arange(len(categories))
width = 0.6

fig, ax = plt.subplots(figsize=(14, 8))

# Bar for 2024
bars = ax.bar(x, spend_2024, width, label='2024', color='orange')

# Diamond markers for 2019 overlapping the bars
ax.scatter(x, spend_2019, color='blue', marker='D', label='2019')

# Labels and formatting
ax.set_xlabel('Categories')
ax.set_ylabel('Share of Spend (%)')
ax.set_title('Share of Online Spend by Country, 2019 vs 2024')
ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=90)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))  # Format y-axis as whole percentages
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
#values used in section 5 are listed in the table below
pivot_country_year_spend_2019_2024

In [ ]:
# Change in online ratio for each mcg from 2019 to 2024
pivot_df = merged_df.pivot(index='mcg', columns='year', values='Online %').fillna(0)

#Calculate absolute and percent change
pivot_df['absolute_change'] = pivot_df[2024] - pivot_df[2019]
pivot_df['percent_change'] = (pivot_df['absolute_change'] / pivot_df[2019]) * 100

pivot_df